# Functions for Courses File

In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
import sqlite3
import courses
from importlib import reload

In [31]:
# Run this if you change the 'courses.py' file but have already created the database in this notebook
reload(courses);

In [4]:
# connect to the database
# use create = True only the very first time running
db = courses.CoursesDB('courses.db', create = True) 

Database created at courses.db


In [5]:
# run this box on your first time connecting to the database
# OR if you change something in the 'courses.py' file related to building the tables
db.drop_all_tables(are_you_sure = True)
db.build_tables()

In [6]:
# load the data for any meets you want to look at
# use db.load_results('link')
db.load_results('https://www.tfrrs.org/results/xc/23538/The_CNU_XC_Invitational')
db.load_results('https://www.tfrrs.org/results/xc/23362/2024_Pirate_Cross_Country_Invitational_')
db.load_results('https://www.tfrrs.org/results/xc/25186/Panorama_Farms_Invitational')
db.load_results('https://tfrrs.org/results/xc/24854/2024_CAA_Cross_Country_Championship')
db.load_results('https://www.tfrrs.org/results/xc/23502/Elon_Phoenix_XC_Invitational_')
db.load_results('https://www.tfrrs.org/results/xc/23503/Friday_Night_Lights_Elon_XC_Opener')
db.load_results('https://www.tfrrs.org/results/xc/25322/NCAA_DI_Southeast_Regional_Cross_Country_Championships')
db.load_results('https://www.tfrrs.org/results/xc/25334/NCAA_Division_I_Cross_Country_Championships')

In [7]:
# all the data together will look something like this
db.run_query('''SELECT * FROM tRunner
JOIN tRaceResult USING(runner_id)
JOIN tRace USING(race_id)''')

,runner_id,name,eligibility,school,race_id,raw_time,time,place,race,date
0,1,Peninah Mutisya,FR-1,Hampton,1,21:13.3,1273.3,1,The CNU XC Invitational,"September 13, 2024"
1,2,Molly Weithman,Freshman,Unattached,1,22:09.4,1329.4,2,The CNU XC Invitational,"September 13, 2024"
2,3,Nia Warren,SR-4,VCU,1,22:14.5,1334.5,3,The CNU XC Invitational,"September 13, 2024"
3,4,Jennifer Tsai,JR-3,William & Mary,1,22:16.8,1336.8,4,The CNU XC Invitational,"September 13, 2024"
4,5,Madelyn Gypson,Freshman,Unattached,1,22:20.3,1340.3,5,The CNU XC Invitational,"September 13, 2024"
...,...,...,...,...,...,...,...,...,...,...
1193,866,Alex Bauer,JR-3,Toledo,8,22:18.6,1338.6,250,NCAA Division I Cross Country Championships,"November 23, 2024"
1194,867,Jayden Harberts,FR-1,Syracuse,8,22:19.7,1339.7,251,NCAA Division I Cross Country Championships,"November 23, 2024"
1195,868,Elizabeth Stockman,SR-4,Texas,8,22:28.9,1348.9,252,NCAA Division I Cross Country Championships,"November 23, 2024"
1196,869,Anna Sentner,SR-4,Florida State,8,22:32.3,1352.3,253,NCAA Division I Cross Country Championships,"November 23, 2024"


In [8]:
# see which races you have loaded
db.see_loaded_races()

,race_id,race,date
0,1,The CNU XC Invitational,"September 13, 2024"
1,2,2024 Pirate Cross Country Invitational,"October 4, 2024"
2,3,Panorama Farms Invitational,"October 19, 2024"
3,4,2024 CAA Cross Country Championship,"November 1, 2024"
4,5,Elon Phoenix XC Invitational,"October 18, 2024"
5,6,Friday Night Lights Elon XC Opener,"September 6, 2024"
6,7,NCAA DI Southeast Regional Cross Country Champ...,"November 15, 2024"
7,8,NCAA Division I Cross Country Championships,"November 23, 2024"


In [13]:
# look up a race by a fragment of the name
db.course_lookup('XC')

,race_id,race,date
0,1,The CNU XC Invitational,"September 13, 2024"
1,5,Elon Phoenix XC Invitational,"October 18, 2024"
2,6,Friday Night Lights Elon XC Opener,"September 6, 2024"


In [21]:
# look up a runner by a fragment of their name
db.runner_lookup('Molly')

,runner_id,name,eligibility,school
0,2,Molly Weithman,Freshman,Unattached
1,64,Molly Williamson,SR-4,Bridgewater (Va.)
2,94,Molly Weithman,FR-1,William & Mary
3,97,Molly Sullivan,JR-3,Davidson
4,290,Molly Jones,SR-4,Coastal Carolina
5,347,Molly Flanagan,FR-1,Richmond
6,521,Molly Dreher,SR-4,Wake Forest
7,543,Molly Lashley,JR-3,Liberty
8,750,Molly Malague,JR-3,Harvard
9,846,Molly Hudson,SR-4,Boston College


In [23]:
# find all meets that two runners ran in together (using their runner_id's)
db.find_races_in_common(1,4)

,race_id,race,date
0,1,The CNU XC Invitational,"September 13, 2024"
1,4,2024 CAA Cross Country Championship,"November 1, 2024"
2,7,NCAA DI Southeast Regional Cross Country Champ...,"November 15, 2024"


In [24]:
# Compare two courses by inputting two race_id's.
# The output is the average time difference in seconds (difference), the ratio of average race times (ratio), and the number of 
#   runners in common between the two courses (NumCompared).

# The first course is used as a comparison point. 'Difference' is the number of seconds faster or slower that the second course
#   averages compared to the first course; 
# ** a negative value for 'difference' means the second course was faster **

# 'Ratio' is the the ratio of course one to course two using average times.
# In other words, the average time from the second course multiplied by 'ratio' should yield the average time from the first course.

# This function only compares times in runners who competed in both meets. The number of runners in common is shown as NumCompared.
db.compare_two_courses(1,2)

,Difference,Ratio,NumCompared
0,16.675,1.011976,12


In [25]:
# Standardize all races to compare times from different courses.
# The input is the race_id that is used as the primary course, and the output is the ratio and time conversions of all other courses
#   to this course
# Faster courses are represented by smaller ratios and negative time conversions
db.conversions(4)

,race_id,race,date,ratio_conversion,time_conversion
0,1,The CNU XC Invitational,"September 13, 2024",1.001394,1.527685
1,2,2024 Pirate Cross Country Invitational,"October 4, 2024",1.014825,20.968421
2,3,Panorama Farms Invitational,"October 19, 2024",0.992819,-10.814341
3,4,2024 CAA Cross Country Championship,"November 1, 2024",1,0
4,5,Elon Phoenix XC Invitational,"October 18, 2024",0.993132,-10.046154
5,6,Friday Night Lights Elon XC Opener,"September 6, 2024",0.643735,-504.08125
6,7,NCAA DI Southeast Regional Cross Country Champ...,"November 15, 2024",1.033221,43.756522
7,8,NCAA Division I Cross Country Championships,"November 23, 2024",1.009171,14.391304


In [35]:
# Use the conversions() function and past results to simulate a virtual meet between specified teams.
# The input is a list of schools the user wants to run a virtual race with and a course the user wants
#   to specify as the primary race.
# The output is the results of the simulated meet if it were run on the same course as the primary race.
db.virtual_race(['William & Mary', 'Elon', 'Davidson', 'Virginia Tech'], 4)

,runner_id,name,school,estimated_time
0,293,Sharon Chepchirchir,Virginia Tech,20:41.8
1,294,Katie Bohlke,Virginia Tech,20:57.9
2,87,Kyra Holland,William & Mary,21:7.07
3,299,Kylie Bonser,Virginia Tech,21:16.5
4,360,Mikayla Jones,Elon,21:16.7
...,...,...,...,...
56,470,Justine Meta,Davidson,24:55.8
57,260,Tina Varis,Elon,25:27.9
58,280,Harrison Grooms,Davidson,26:13.3
59,434,Alme Jordaan,Elon,27:12.1
